In [7]:
%pip install langchain pandas chromadb sentence_transformers unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.5 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 48.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 70.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 40.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 62.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 21.5 MB/s eta 0:00:0

In [11]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pathlib

domain = "www.cnnbrasil.com.br"
DATA_PATH = pathlib.Path(f"text/{domain}")

def load_documents(): 
    loader = DirectoryLoader(DATA_PATH, glob="*.txt")
    return loader.load()

def split_texts(documents):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=500,
        length_function=len, 
        add_start_index=True
    )
    return splitter.split_documents(documents)


In [23]:
# Persistance 
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import HuggingFaceEmbeddings


CHROMA_PATH = pathlib.Path("chroma")
embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

def save_documents(documents): 
    if not CHROMA_PATH.exists():
        CHROMA_PATH.mkdir()

    db = Chroma.from_documents(documents=documents, embedding=embedder, persist_directory=str(CHROMA_PATH))
    db.persist() 

def generate_data_store(): 
    documents = load_documents()
    chunks = split_texts(documents)
    save_documents(chunks)

/home/batman/workspace/webcrawl-class/.venv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [24]:
generate_data_store()

/home/batman/workspace/webcrawl-class/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [27]:
def query_chroma(query: str, limit: int = 5): 
    db = Chroma(persist_directory=str(CHROMA_PATH), embedding_function=embedder)
    return db.similarity_search_with_relevance_scores(query, k=limit)

query_chroma("Bolsonaro")

[(Document(page_content='Câmara dos Deputados   Brasileiros levaram ao Comitê de Assuntos Estrangeiros alegações de censura e violações da liberdade de expressão                07/05/2024 às 21:00    Compartilhe:                                        Uma comitiva de nomes conservadores e ligados ao ex-presidente Jair Bolsonaro viajou à capital norte-americana, Washington DC, para participar de uma audiência sobre o Brasil na Câmara dos Deputados.O grupo alega sofrer censura por parte de autoridades brasileiras, principalmente pelo ministro Alexandre de Moraes, do Supremo Tribunal Federal (STF).Foram quatro integrantes do painel ouvidos: o ex-apresentador da Jovem Pan, Paulo Figueiredo; o jornalista norte-americano Michael Shellenberger, responsável pela divulgação dos documentos sobre o Twitter no Brasil; Chris Pavlovski, CEO da plataforma de vídeos Rumble, popular entre conservadores, e Fábio de Sá e Silva, professor de estudos brasileiros na Universidade de Oklahoma.Fábio de Sá e Si